## 캐싱(Caching)

LangChain은 LLM을 위한 선택적 캐싱 레이어를 제공합니다.

이는 두 가지 이유로 유용합니다.

- 동일한 완료를 여러 번 요청하는 경우 LLM 공급자에 대한 **API 호출 횟수를 줄여 비용을 절감**할 수 있습니다.
- LLM 제공업체에 대한 **API 호출 횟수를 줄여 애플리케이션의 속도를 높일 수** 있습니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [4]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


모델과 프롬프트를 생성합니다


In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 모델을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o-mini")

# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template("{country} 에 대해서 200자 내외로 요약해줘")

# 체인을 생성합니다.
chain = prompt | llm

In [7]:
%%time 
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 한반도의 국가로, 북한과 남한으로 나뉘어 있습니다. 서울은 남한의 수도이며, 경제, 문화, 기술에서 세계적인 영향력을 미칩니다. 한국은 역사적으로 고대 왕국에서 시작해 다양한 문화와 전통을 발전시켜왔습니다. 한국 음식, K-POP, 드라마 등은 글로벌한 인기를 끌고 있으며, 정보통신 기술과 제조업에서 강력한 경쟁력을 보이고 있습니다.
CPU times: total: 0 ns
Wall time: 5.47 s


## InMemoryCache

인메모리 캐시를 사용하여 동일 질문에 대한 답변을 저장하고, 캐시에 저장된 답변을 반환합니다.

In [8]:
%%time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

# 인메모리 캐시를 사용합니다.
set_llm_cache(InMemoryCache())

# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 반도 국가로, 한반도와 그 주변의 섬들로 구성되어 있습니다. 서울이 수도이며, 인구는 약 5천만 명입니다. 풍부한 역사와 문화, K-팝과 한국 드라마로 세계적으로 알려져 있습니다. 경제는 첨단 기술과 제조업 중심이며, 교육 수준이 높습니다. 김치와 비빔밥 등 다양한 전통 음식이 있으며, 독특한 언어와 문자(한글)를 가지고 있습니다.
CPU times: total: 0 ns
Wall time: 2.44 s


In [9]:
%%time
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)

한국은 동아시아에 위치한 반도 국가로, 한반도와 그 주변의 섬들로 구성되어 있습니다. 서울이 수도이며, 인구는 약 5천만 명입니다. 풍부한 역사와 문화, K-팝과 한국 드라마로 세계적으로 알려져 있습니다. 경제는 첨단 기술과 제조업 중심이며, 교육 수준이 높습니다. 김치와 비빔밥 등 다양한 전통 음식이 있으며, 독특한 언어와 문자(한글)를 가지고 있습니다.
CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
%%time
from langchain.globals import set_llm_cache
from langchain_community.cache import InMemoryCache
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI

# 캐시 설정
set_llm_cache(InMemoryCache())

# 체인 구성
prompt = ChatPromptTemplate.from_template("한국의 수도는 무엇인가요?")
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = prompt | llm | StrOutputParser()

print("=== 첫 번째 실행 ===")
response1 = chain.invoke({})
print(f"응답: {response1}")

print("\n=== 두 번째 실행 ===")
response2 = chain.invoke({})
print(f"응답: {response2}")

=== 첫 번째 실행 ===
응답: 한국의 수도는 서울입니다.

=== 두 번째 실행 ===
응답: 한국의 수도는 서울입니다.
CPU times: total: 31.2 ms
Wall time: 1.1 s


## SQLite Cache


In [ ]:
from langchain_community.cache import SQLiteCache
from langchain_core.globals import set_llm_cache
import os

# 캐시 디렉토리를 생성합니다.
if not os.path.exists("cache"):
    os.makedirs("cache")

# SQLiteCache를 사용합니다.
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

In [ ]:
%%time 
# 체인을 실행합니다.
response = chain.invoke({"country": "한국"})
print(response.content)